In [4]:
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [9]:
# Загрузим данные
df = pd.read_csv('C:/Users/HP/Desktop/car_battery.csv')

In [63]:
df.iloc[0]

crawling_start                                     2024-03-18T01:07:28
crawling_date                                      2024-03-18T01:07:55
source                                                   Яндекс Маркет
name                 Тяговый аккумулятор RuTrike 6-DZF-12 (12V13A/H...
description          Тяговый аккумулятор RuTrike 6-DZF-12 (12V13A/H...
img_url                                                            NaN
rating                                                             5.0
ratingsNumber                                                        2
url                  https://market.yandex.ru/product--moto-akkumul...
price                                                                0
salePrice                                                            0
polarity                                                        прямая
voltage                                                             12
inrushCurrent                                                        2
capaci

In [11]:
df.columns

Index(['crawling_start', 'crawling_date', 'source', 'name', 'description',
       'img_url', 'rating', 'ratingsNumber', 'url', 'price', 'salePrice',
       'polarity', 'voltage', 'inrushCurrent', 'capacity', 'typeOfVehicle',
       'batteryTechnology', 'isServiced', 'features', 'klemmy', 'caseType',
       'batteryMarking', 'additional', 'length', 'width', 'height', 'weight',
       'serviceLife', 'warrantyPeriod'],
      dtype='object')

In [64]:
# Предположим, что у нас есть две батареи, которые мы хотим сравнить
battery1 = df.iloc[31]  # Определяем два товара для сравнения
battery2 = df.iloc[40]

# Словарь, представляющий важность каждого атрибута (нужно предварительно определить)
weights = {
    'name': 1.0,
    'voltage': 0.8,
    'batteryTechnology': 0.8,
    'klemmy': 0.8,
    'height': 0.6,
    'weight': 0.6,
    'length': 0.6,
    'width': 0.6,
    'description': 0.3,
    # Добавьте все сравниваемые атрибуты
}

In [29]:
str_columns = ('name', 'batteryTechnology', 'klemmy', 'description')

In [61]:
# Функция для сравнения двух батарей
def compare_batteries(battery1, battery2):
    total_score = 0
    
    for attribute, weight in weights.items():
        value1 = battery1[attribute]
        value2 = battery2[attribute]
        
        # Если одно из значений пропущено, это значение должно быть учетено с меньшим весом
        if pd.isnull(value1) or pd.isnull(value2):
            weight *= 0.5
        
        # Если это строка, используем нечеткую схожесть
        if attribute in str_columns:
            # Представляем все буквы в нижнем регистре
            value1 = str(value1).lower()
            value2 = str(value2).lower()
            score = fuzz.ratio(value1, value2) / 100.0  # Эта функция вернет число от 0 до 100
        else:
            # Используем абсолютную разницу для численных значений
            # Это не идеальная метрика, но ее можно использовать в качестве простого примера
            max_value = df[df[attribute] != -1.0][attribute].max() 
            min_value = df[df[attribute] != -1.0][attribute].min()
            if max_value == min_value:  
                score = 1.0
            else:
                score = 1.0 - abs(value1 - value2) / (max_value - min_value)
        
        # Прибавляем к общему счету, умножая на вес атрибута
        total_score += weight * score
    
    return total_score

In [62]:
# Получаем оценку сходства для двух батарей
similarity = compare_batteries(battery1, battery2)
print(similarity)

5.699999999999999


In [90]:
dic = {}
for i in range (0, 100):
    b2 = df.iloc[i]
    s = compare_batteries(battery1, b2)
    dic[b2["name"]] = s
    print(f'bat: {b2["name"]} sim: {s}')

bat: Тяговый аккумулятор RuTrike 6-DZF-12 (12V13A/H C3) sim: 2.721908143444876
bat: Аккумулятор EXTRA START 62e 6СТ-62N L+ (L2), 242x175x190, прямая полярность, 62 Ач sim: 3.9625748101115428
bat: Автомобильный аккумулятор премиум класса WPR 6ст- 50 VL прямая полярность (низ.) sim: 3.836065820578758
bat: Аккумулятор для грузовиков FireBall 6СТ-60NR, 242х175х190 sim: 3.900333333333333
bat: Аккумулятор автомобильный AGM IQ Energy Standart 12v 80AH 820 А обратная полярность 315x175x190 стартерный акб для автомобиля, машины sim: 2.6372414767782097
bat: Автомобильный аккумулятор Varta Vst Стандарт 6СТ-60 Ah (560 300 054) обратная полярность sim: 3.867
bat: Аккумулятор forse 60ач en640/600 242х175х175 низкий обр/п sim: 2.2584839822919975
bat: Автомобильный аккумулятор FIRE BALL 6СТ-60 (1) N (арт. 560107020) sim: 3.9713333333333334
bat: Аккумулятор Аком + EFB 60Е (580 A), 242х175х190, полярность обратная sim: 3.6556666666666664
bat: Аккумулятор для мотоцикла 12в гелевый 9 Ah 135 A прямая поляр

In [92]:
sorted_dic = {k : v for k, v in sorted(dic.items(), key = lambda item: item[1], reverse = True)}

In [93]:
print(sorted_dic)

{'Автомобильный аккумулятор Extra Start 6СТ-60N R+ (L2), 242х175х190': 5.068, 'Аккумулятор Extra Start 6СТ-100N R+ (L5), 353х175х190': 4.996888700313926, 'Автомобильный аккумулятор Extra Start 6СТ-60N L+ (L2), 242х175х190': 4.497, 'Автомобильный аккумулятор VARTA Silver Dynamic D15 (563 400 061), 242х175х190': 4.454111111111112, 'Автомобильный аккумулятор Mutlu SFB 3 (L5.100.090.A), 353х175х190': 4.448555366980594, 'Автомобильный аккумулятор TYUMEN BATTERY PREMIUM 6СТ-77L 670А п.п., 278х175х190': 4.373555494396108, 'Автомобильный аккумулятор премиум класса SF SONIC 6СТ-45 Ач тонк. кл': 4.349742230584155, 'Автомобильный аккумулятор Topla Energy 108066, 242х175х190': 4.083666666666667, 'Автомобильный аккумулятор TYUMEN BATTERY STANDARD 6CT-60L 550А о.п., 242х175х190': 4.065111111111111, 'Автомобильный аккумулятор FORSE 6СТ-60 VLR (0) (арт. 560108050)': 4.013666666666667, 'Автомобильный аккумулятор Solite CMF50AL, 207х175х190, полярность обратная': 3.993777718317204, 'Автомобильный аккуму